# Using Versioning in S3 Bucket

This notebook is a detailed, interactive tutorial for learning.

Reference:

- [Using Versioning in S3 Bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/Versioning.html)

## 01. Prepare Your Playground

First, we need to prepare our development environment for a better learning experience.

- an AWS CLI Profile with access to S3, it should have S3 full access and STS get-caller-identity permission.
- a bucket with versioning turned on (we will create it soon)
- Installed the following python library:
    - [boto_session_manager](https://pypi.org/project/boto_session_manager/): boto3 session management made easy
    - [s3pathlib](https://pypi.org/project/s3pathlib/): s3 manipulation made easy
    - [rich](https://pypi.org/project/s3pathlib/): for pretty print

In [1]:
# Enter your AWS Profile here
aws_profile = "awshsh_app_dev_us_east_1"

In [47]:
from rich import print as rprint
from pprint import pprint

def rprint_response(res: dict):
    """
    Pretty print boto3 API response
    """
    if "ResponseMetadata" in res:
        res.pop("ResponseMetadata")
    rprint(res)

In [32]:
from boto_session_manager import BotoSesManager
from s3pathlib import S3Path, context

bsm = BotoSesManager(profile_name=aws_profile)
context.attach_boto_session(bsm.boto_ses)

bucket = f"{bsm.aws_account_id}-{bsm.aws_region}-learn-s3-versioning"

# Create the bucket and turn on versioning
def is_bucket_exists() -> bool:
    try:
        bsm.s3_client.head_bucket(Bucket=bucket)
        return True
    except bsm.s3_client.exceptions.ClientError as e:
        return False

print("Try to create the bucket ...")
if is_bucket_exists() is False:
    kwargs = dict(Bucket=bucket)
    if bsm.aws_region != "us-east-1":
        kwargs["CreateBucketConfiguration"] = dict(LocationConstraint=bsm.aws_region)
    bsm.s3_client.create_bucket(**kwargs)
    print("done bucket is created")
else:
    print("bucket already exists")

print("Try to turn on bucket versioning ...")
response = bsm.s3_client.get_bucket_versioning(
    Bucket=bucket,
)
if "Status" in response: # versioning is already enabled or suspended
    pass
else: # versioning is not enabled
    bsm.s3_client.put_bucket_versioning(
        Bucket=bucket,
        VersioningConfiguration=dict(
            Status="Enabled",
        )
    )
print("done")

# verify if bucket versioning is enabled
response = bsm.s3_client.get_bucket_versioning(
    Bucket=bucket,
)
rprint_response(response)
print(f"preview S3 bucket: {S3Path(bucket).console_url}")

Try to create the bucket ...
bucket already exists
Try to turn on bucket versioning ...
done


{'Status': 'Enabled'}

preview S3 bucket: https://console.aws.amazon.com/s3/buckets/807388292768-us-east-1-learn-s3-versioning?tab=objects


## 02. Put and Get

With versioning, everytime you invoke [put_object](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/put_object.html) API, a new version of the object will be created. And everytime you invoke [get_object](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/get_object.html) API, you can get the latest version of the object.

Let's test it. First, we create a new object in a bucket with versioning turned on. It is also the first version of this object.

Reference:

- [Adding object](https://docs.aws.amazon.com/AmazonS3/latest/userguide/AddingObjectstoVersioningEnabledBuckets.html)
- [Retrieving object versions from a versioning-enabled bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/RetrievingObjectVersions.html)


In [33]:
print("Create a new object, which is also the first version of this object ...")
s3path = S3Path(bucket, "test.txt")
res = bsm.s3_client.put_object(
    Bucket=s3path.bucket,
    Key=s3path.key,
    Body="content v1",
)
rprint_response(res)

v1 = res["VersionId"]
print(f"The version id (v1) = {v1}")

Create a new object, which is also the first version of this object ...


{
    'ETag': '"99acfefa036b6b3bf0949f1d9ba8acb2"',
    'ServerSideEncryption': 'AES256',
    'VersionId': 'qxS7XGuBGR5b8.12S4lrshKOqY.5r6J8'
}

The version id (v1) = qxS7XGuBGR5b8.12S4lrshKOqY.5r6J8


Then we can immediately get the object. By default, the latest version is returned.

In [34]:
print("Get the object ...")
res = bsm.s3_client.get_object(Bucket=s3path.bucket, Key=s3path.key)
rprint_response(res)

content = res["Body"].read().decode("utf-8")
assert content == "content v1"
print("Content = {}".format(content))

v = res["VersionId"]
assert v == v1
print("The version id (v1) = {}".format(v))

Get the object ...


{
    'AcceptRanges': 'bytes',
    'LastModified': datetime.datetime(2023, 4, 16, 2, 30, 20, tzinfo=tzutc()),
    'ContentLength': 10,
    'ETag': '"99acfefa036b6b3bf0949f1d9ba8acb2"',
    'VersionId': 'qxS7XGuBGR5b8.12S4lrshKOqY.5r6J8',
    'ContentType': 'binary/octet-stream',
    'ServerSideEncryption': 'AES256',
    'Metadata': {},
    'Body': <botocore.response.StreamingBody object at 0x106b952e0>
}

Content = content v1
The version id (v1) = qxS7XGuBGR5b8.12S4lrshKOqY.5r6J8


Then we put a new content to this object, which creates a new version of the object. Note that you cannot overwrite an existing version, because the versioning system is designed to ensure immutability. That's why the ``put_object`` API doesn't have argument ``VersionId``.

In [35]:
print("Put a new version of the object ...")
res = bsm.s3_client.put_object(
    Bucket=s3path.bucket,
    Key=s3path.key,
    Body="content v2",
)
v2 = res["VersionId"]
print(f"The version id (v2) = {v2}")

Put a new version of the object ...
The version id (v2) = k59flX7WM58iS.DcZKdVWl46l8wi6_Sw


Now, let's get the object again. By default, it get the latest version, and we can see that the version id is different from the previous one.

In [36]:
print("Get the object again, now it should be the content of v2 ...")
res = bsm.s3_client.get_object(Bucket=s3path.bucket, Key=s3path.key)

content = res["Body"].read().decode("utf-8")
assert content == "content v2"
print(f"Content = {content}")

v = res["VersionId"]
assert v == v2
print(f"The version id (v2) = {v}")

Get the object again, now it should be the content of v2 ...
Content = content v2
The version id (v2) = k59flX7WM58iS.DcZKdVWl46l8wi6_Sw


We can explicitly get a historical version using version id

In [37]:
print("Explicitly get the v1 version ...")
res = bsm.s3_client.get_object(Bucket=s3path.bucket, Key=s3path.key, VersionId=v1)

content = res["Body"].read().decode("utf-8")
assert content == "content v1"
print(f"Content = {content}")

v = res["VersionId"]
assert v == v1
print(f"The version id = {v}")

Explicitly get the v1 version ...
Content = content v1
The version id = qxS7XGuBGR5b8.12S4lrshKOqY.5r6J8


We can also use the [list_object_versions](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/list_object_versions.html) API to list all the historical versions of an object. It will return in order of last modified time, from the latest to the oldest.

In [39]:
print("List all historical versions")
res = bsm.s3_client.list_object_versions(Bucket=s3path.bucket, Prefix=s3path.key)
rprint_response(res)

n_versions = len(res["Versions"])
print(f"Number of versions = {n_versions}")

List all historical versions


{
    'IsTruncated': False,
    'KeyMarker': '',
    'VersionIdMarker': '',
    'Versions': [
        {
            'ETag': '"96221cd7501efb4f0ce38d99cfb133e5"',
            'Size': 10,
            'StorageClass': 'STANDARD',
            'Key': 'test.txt',
            'VersionId': 'k59flX7WM58iS.DcZKdVWl46l8wi6_Sw',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 4, 16, 2, 30, 21, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"99acfefa036b6b3bf0949f1d9ba8acb2"',
            'Size': 10,
            'StorageClass': 'STANDARD',
            'Key': 'test.txt',
            'VersionId': 'qxS7XGuBGR5b8.12S4lrshKOqY.5r6J8',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 16, 2, 30, 20, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        }
    ],
    'Name': '807388292768-us-east-1-learn-s3-versioning',
    'Prefix': 'test.txt',
    'MaxKeys': 1000,
    'EncodingType': 'url'
}

Number of versions = 2


## 03. Delete

There is only one 'delete' API [
delete_object](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/delete_object.html), but there are two ways to use it.

First way is that, if you call the ``delete_object`` API without giving the ``VersionId``, then it is a **regular delete**. It will mark the latest version of the object as "Deleted", and create a ``Marker``. After that, if you call ``get_object`` API without giving the ``VersionId``, it will return 404 error. Because it will find the latest version of this object, and see that it is marked as "Deleted". However, the content and the historical versions are still there, you can still get them by explicitly giving the ``VersionId``.

``Marker`` itself is not an attribute of a version, it is actually a tiny object with ``VersionId``. Note that this ``VersionId`` is the identifier of the marker, it is NOT the ``VersionId`` of the object version you deleted. This maker ``VersionId`` can be used to recover the deleted version by deleting it via the ``delete_object``.

Secondly way is that, if you call the ``delete_object`` API with a ``VersionId``, then it deletes the specific version. This method can also be used to delete a ``Maker``.

Next, we would like to test the deletion behavior. First, we put 3 more versions, so we have more versions to test with.

In [40]:
print("put 3 more versions for testing, we should have 5 versions in total.")
res = bsm.s3_client.put_object(Bucket=s3path.bucket, Key=s3path.key, Body="content v3")
v3 = res["VersionId"]

res = bsm.s3_client.put_object(Bucket=s3path.bucket, Key=s3path.key, Body="content v4")
v4 = res["VersionId"]

res = bsm.s3_client.put_object(Bucket=s3path.bucket, Key=s3path.key, Body="content v5")
v5 = res["VersionId"]

print(f"v3 = {v3}")
print(f"v4 = {v4}")
print(f"v5 = {v5}")

put 3 more versions for testing, we should have 5 versions in total.
v3 = c3VQKiaUgZMqS0y21hHahitBAsT.b9Td
v4 = lnCOajuiT8mQoQzyMurku8b5vTAgO2_c
v5 = 3WXcamR3xEOSmEU6.SyU7eCnWIDe5f0c


Now we try to delete the object. What happens is that the latest object version is marked as deleted, but the content and the historical versions are still there.

In [41]:
print("Delete the object, it marks the latest version as 'Deleted'")
res = bsm.s3_client.delete_object(Bucket=s3path.bucket, Key=s3path.key)
rprint_response(res)

m5 = res["VersionId"]
print(f"Marker Id (m5) = {m5}")

Delete the object, it marks the latest version as 'Deleted'


{'DeleteMarker': True, 'VersionId': 'VWg5vwE3xD4pQ.gWkDaVngscosGMy.EI'}

Marker Id (m5) = VWg5vwE3xD4pQ.gWkDaVngscosGMy.EI


Then, let's try to get the object. S3 will get the latest version of this object, and find out it is marked as deleted, so it will return a 404 error.

In [43]:
print("Get the object, it should returns a 404 error")
res = bsm.s3_client.get_object(Bucket=s3path.bucket, Key=s3path.key)

Get the object, it should returns a 404 error


NoSuchKey: An error occurred (NoSuchKey) when calling the GetObject operation: The specified key does not exist.

In [49]:
print("List all historical versions ...")
print("Now you should see a new field 'DeleteMarkers' and it has a marker object")
res = bsm.s3_client.list_object_versions(Bucket=s3path.bucket, Prefix=s3path.key)
rprint_response(res)
# pprint(res)

List all historical versions ...
Now you should see a new field 'DeleteMarkers' and it has a marker object


{
    'IsTruncated': False,
    'KeyMarker': '',
    'VersionIdMarker': '',
    'Versions': [
        {
            'ETag': '"d72dcc817399f106ad64eb7bb3089006"',
            'Size': 10,
            'StorageClass': 'STANDARD',
            'Key': 'test.txt',
            'VersionId': '3WXcamR3xEOSmEU6.SyU7eCnWIDe5f0c',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 16, 2, 39, 52, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"562a103d5a81ac473581c75552991f86"',
            'Size': 10,
            'StorageClass': 'STANDARD',
            'Key': 'test.txt',
            'VersionId': 'lnCOajuiT8mQoQzyMurku8b5vTAgO2_c',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 16, 2, 39, 52, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"83f457d8c45b9f866c01c7c39ea0d917"',
            'Size': 10,
            'StorageClass': 'STANDARD',
            'Key': 'test.txt',
            'VersionId': 'c3VQKiaUgZMqS0y21hHahitBAsT.b9Td',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 16, 2, 39, 52, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"96221cd7501efb4f0ce38d99cfb133e5"',
            'Size': 10,
            'StorageClass': 'STANDARD',
            'Key': 'test.txt',
            'VersionId': 'k59flX7WM58iS.DcZKdVWl46l8wi6_Sw',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 16, 2, 30, 21, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"99acfefa036b6b3bf0949f1d9ba8acb2"',
            'Size': 10,
            'StorageClass': 'STANDARD',
            'Key': 'test.txt',
            'VersionId': 'qxS7XGuBGR5b8.12S4lrshKOqY.5r6J8',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 16, 2, 30, 20, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        }
    ],
    'DeleteMarkers': [
        {
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            },
            'Key': 'test.txt',
            'VersionId': 'VWg5vwE3xD4pQ.gWkDaVngscosGMy.EI',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 4, 16, 2, 41, 10, tzinfo=tzutc())
        }
    ],
    'Name': '807388292768-us-east-1-learn-s3-versioning',
    'Prefix': 'test.txt',
    'MaxKeys': 1000,
    'EncodingType': 'url'
}

In [ ]:

# content = res["Body"].read().decode("utf-8")
# v = res["VersionId"]
#
# assert content == "v4"
# assert v == v4
#
# print(f"Content = {content}")
# print(f"Version Id = {v}")

In [80]:
res = bsm.s3_client.get_object(Bucket=bucket, Key=key, VersionId=v4)
rprint(res)

{
    'ResponseMetadata': {
        'RequestId': '9VPZY36D5TVX8DF6',
        'HostId': 'U53YbOgyAM+oW9yxxOQl56tY0cAPnK0174esyBx1m/0+nqRMMxJgzA+AJ3rMd/l+EsvBJo9Ztuk=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'U53YbOgyAM+oW9yxxOQl56tY0cAPnK0174esyBx1m/0+nqRMMxJgzA+AJ3rMd/l+EsvBJo9Ztuk=',
            'x-amz-request-id': '9VPZY36D5TVX8DF6',
            'date': 'Sat, 15 Apr 2023 21:39:16 GMT',
            'last-modified': 'Sat, 15 Apr 2023 21:35:39 GMT',
            'etag': '"5ed3a3ff8e5c29979502b9030e8518e0"',
            'x-amz-server-side-encryption': 'AES256',
            'x-amz-version-id': 'zufkTCbIA7N7DE0SPXS6AySBZzvRV5Oj',
            'accept-ranges': 'bytes',
            'content-type': 'binary/octet-stream',
            'server': 'AmazonS3',
            'content-length': '2'
        },
        'RetryAttempts': 0
    },
    'AcceptRanges': 'bytes',
    'LastModified': datetime.datetime(2023, 4, 15, 21, 35, 39, tzinfo=tzutc()),
    'ContentLength': 2,
    'ETag': '"5ed3a3ff8e5c29979502b9030e8518e0"',
    'VersionId': 'zufkTCbIA7N7DE0SPXS6AySBZzvRV5Oj',
    'ContentType': 'binary/octet-stream',
    'ServerSideEncryption': 'AES256',
    'Metadata': {},
    'Body': <botocore.response.StreamingBody object at 0x1080abca0>
}

Now we can list all version again

In [59]:
res1 = bsm.s3_client.list_object_versions(Bucket=bucket, Prefix=key)
rprint(res1)

{
    'ResponseMetadata': {
        'RequestId': '9E6489A8KCB77S8T',
        'HostId': 'TfIuLv+2YBlabS5Pzp6mmCgQIQ3Ylds9BOzb5O+trF095DJYFeCafWdTwEeZ0KL6K5QvlRIbBF8=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'TfIuLv+2YBlabS5Pzp6mmCgQIQ3Ylds9BOzb5O+trF095DJYFeCafWdTwEeZ0KL6K5QvlRIbBF8=',
            'x-amz-request-id': '9E6489A8KCB77S8T',
            'date': 'Sat, 15 Apr 2023 21:30:23 GMT',
            'content-type': 'application/xml',
            'transfer-encoding': 'chunked',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    },
    'IsTruncated': False,
    'KeyMarker': '',
    'VersionIdMarker': '',
    'Versions': [
        {
            'ETag': '"4b6df01946f4919a3f5af8c1f0f9c3c5"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'nAPU_RhaanKQt7UaDQfz02dqvyjKFj8I',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 27, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"5ed3a3ff8e5c29979502b9030e8518e0"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'Kb0aorSWLcS_3N2knqz_9zwufm6F8xEC',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 27, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"43a03299a3c3fed3d8ce7b820f3aca81"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': '8h9fqea4xGhze6eugeHgNbT5Jccozr7y',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 27, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"1b267619c4812cc46ee281747884ca50"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'pKCvdIbB47hE3sAFZjDCqt.xqheRjkDD',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 26, 55, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"6654c734ccab8f440ff0825eb443dc7f"',
            'Size': 2,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'dgxxL.IhdRvXQ85eTPi.Py82eu_kEN57',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 26, 53, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        }
    ],
    'DeleteMarkers': [
        {
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            },
            'Key': 'projects/learn-s3-versioning/business-report.txt',
            'VersionId': 'nSerOG9XapELdzYL9FAYflgBOyhHRM6f',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 

In [24]:
_ = bsm.s3_client.put_object(
    Bucket=bucket,
    Key=f"{prefix}/folder/file1.txt",
    Body="file1-v1",
)
_ = bsm.s3_client.put_object(
    Bucket=bucket,
    Key=f"{prefix}/folder/file1.txt",
    Body="file1-v2",
)
_ = bsm.s3_client.put_object(
    Bucket=bucket,
    Key=f"{prefix}/folder/file2.txt",
    Body="file2-v1",
)
_ = bsm.s3_client.put_object(
    Bucket=bucket,
    Key=f"{prefix}/folder/file2.txt",
    Body="file2-v2",
)

In [25]:
res = bsm.s3_client.list_object_versions(
    Bucket=bucket,
    Prefix=f"{prefix}/folder/",
)
rprint(res)

{
    'ResponseMetadata': {
        'RequestId': 'NKKHZYCPWWYZ1J32',
        'HostId': 'EcVK6lDcEj3hyaERLo0bD+U8h9qK7x1sL3a8Xub0zre2n737mn6yHg2xLbWer2aTSme3XtCRse0=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'EcVK6lDcEj3hyaERLo0bD+U8h9qK7x1sL3a8Xub0zre2n737mn6yHg2xLbWer2aTSme3XtCRse0=',
            'x-amz-request-id': 'NKKHZYCPWWYZ1J32',
            'date': 'Sat, 15 Apr 2023 21:10:37 GMT',
            'content-type': 'application/xml',
            'transfer-encoding': 'chunked',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    },
    'IsTruncated': False,
    'KeyMarker': '',
    'VersionIdMarker': '',
    'Versions': [
        {
            'ETag': '"9131f6503b12d73dcfb9f641f517da04"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': '9yWglLuLZXdWT7AT1KllqojQuJhScFIE',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 29, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'tV4U5RRJMDLEwfKxQAVe_r74KzeUdWLC',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 29, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"9131f6503b12d73dcfb9f641f517da04"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'H6_lF33evUYLZsIoxd24kl7hoH0xXbuU',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 14, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': '8AUw3qB1poFyPlhqHPN55jdHWqzrzHtT',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 14, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'wboEjS_ceV0C4s.K4Twx1hDPkhGgBGXj',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 9, 54, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"20a8caaee5116a21cfcdbf2fbd5bc602"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file2.txt',
            'VersionId': 'IghSoYgJwy_JiYnsnx1.PA0WJSCA4ilQ',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 29, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+a

In [26]:
res = bsm.s3_client.list_object_versions(
    Bucket=bucket,
    Prefix=f"{prefix}/folder/file1.txt",
)
rprint(res)

{
    'ResponseMetadata': {
        'RequestId': '3J15A6YHFE3ZMJPD',
        'HostId': 'GSnmcn1hQxDKLpdmDG9pX4RFYFxV6P+D8JOaBm3m8lsltRLLwkLtWLLCRju3iSVLsyMuFSdJ/fs=',
        'HTTPStatusCode': 200,
        'HTTPHeaders': {
            'x-amz-id-2': 'GSnmcn1hQxDKLpdmDG9pX4RFYFxV6P+D8JOaBm3m8lsltRLLwkLtWLLCRju3iSVLsyMuFSdJ/fs=',
            'x-amz-request-id': '3J15A6YHFE3ZMJPD',
            'date': 'Sat, 15 Apr 2023 21:10:46 GMT',
            'content-type': 'application/xml',
            'transfer-encoding': 'chunked',
            'server': 'AmazonS3'
        },
        'RetryAttempts': 0
    },
    'IsTruncated': False,
    'KeyMarker': '',
    'VersionIdMarker': '',
    'Versions': [
        {
            'ETag': '"9131f6503b12d73dcfb9f641f517da04"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': '9yWglLuLZXdWT7AT1KllqojQuJhScFIE',
            'IsLatest': True,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 29, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'tV4U5RRJMDLEwfKxQAVe_r74KzeUdWLC',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 29, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"9131f6503b12d73dcfb9f641f517da04"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'H6_lF33evUYLZsIoxd24kl7hoH0xXbuU',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 14, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': '8AUw3qB1poFyPlhqHPN55jdHWqzrzHtT',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 10, 14, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        },
        {
            'ETag': '"112e3e27b3f77487cbed73cf1deacb5f"',
            'Size': 8,
            'StorageClass': 'STANDARD',
            'Key': 'projects/learn-s3-versioning/folder/file1.txt',
            'VersionId': 'wboEjS_ceV0C4s.K4Twx1hDPkhGgBGXj',
            'IsLatest': False,
            'LastModified': datetime.datetime(2023, 4, 15, 21, 9, 54, tzinfo=tzutc()),
            'Owner': {
                'DisplayName': 'sanhehu+awshsh-app-dev',
                'ID': 'd8468a5d68b2a6e29f0436ec9b64ed212e2b3a272d15429fbfc58a3c35ff4bf7'
            }
        }
    ],
    'Name': '807388292768-us-east-1-data-versioned',
    'Prefix': 'projects/learn-s3-versioning/folder/file1.txt',
    'MaxKeys': 1000,
    'EncodingType': 'url'
}